<a href="https://colab.research.google.com/github/alexlaurence/DoReMi/blob/master/DoReMi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DoReMi (v0.1.0-alpha)

By Alexander Laurence (alexander.adamlaurence@gmail.com)

http://celestial.tokyo/~AlexLaurence/

DoReMi is a tool for blind users to search job listings from by synthesising job listing RSS feeds from Scout.co.kr into speech with the help of Google's Text-to-Speech API powered by a WaveNet model.

**Why Google Text-to-Speech?**

The Text-to-Speech API creates raw audio data of natural, human speech. That is, it creates audio that sounds like a person talking. When you send a synthesis request to the Text-to-Speech API, you must specify a voice that 'speaks' the words.

**Why WaveNet?**

A WaveNet generates speech that sounds more natural than other text-to-speech systems. It synthesizes speech with more human-like emphasis and inflection on syllables, phonemes, and words. On average, a WaveNet produces speech audio that people prefer over other text-to-speech technologies.

Unlike most other text-to-speech systems, a WaveNet model creates raw audio waveforms from scratch. The model uses a neural network that has been trained using a large volume of speech samples. During training, the network extracts the underlying structure of the speech, such as which tones follow each other and what a realistic speech waveform looks like. When given a text input, the trained WaveNet model can generate the corresponding speech waveforms from scratch, one sample at a time, with up to 24,000 samples per second and seamless transitions between the individual sounds.

## Installation

In [0]:
!pip install feedparser
!pip install --upgrade google-cloud-speech

In [0]:
import feedparser
import re, os, glob, shutil

from google.cloud import texttospeech
from google.oauth2 import service_account

from IPython.display import Audio
from scipy.io import wavfile

In [0]:
#set API Credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='/content/My First Project-a8dd55fddd5e.json'

## Try it out!

In [0]:
def start():
  
  credentials = service_account.Credentials.from_service_account_file('/content/My First Project-a8dd55fddd5e.json')
  client = texttospeech.TextToSpeechClient(credentials=credentials)
  
  # set xml url as strings
  seoul = "http://feeds.feedburner.com/co/clsY"
  gangwon = "http://feeds.feedburner.com/co/Utav"
  gyeonggi = "http://feeds.feedburner.com/co/lUKP"
  gyeongnam = "http://feeds.feedburner.com/co/rMCwN"
  gyeongbuk = "http://feeds.feedburner.com/co/Dhwm"
  gwangju = "http://feeds.feedburner.com/co/LXwe"
  daegu = "http://feeds.feedburner.com/co/Bcsg"
  daejeon = "http://feeds.feedburner.com/co/zBIF"
  busan = "http://feeds.feedburner.com/co/dkiFn"
  ulsan = "http://feeds.feedburner.com/co/LFJz"
  incheon = "http://feeds.feedburner.com/co/EkCu"
  jeonnam = "http://feeds.feedburner.com/co/wjlNw"
  jeonbuk = "http://feeds.feedburner.com/co/ZaXx"
  jeju = "http://feeds.feedburner.com/co/viTpC"
  chungnam = "http://feeds.feedburner.com/co/yIBb"
  chungbuk = "http://feeds.feedburner.com/co/aXuFR"
  national = "http://feeds.feedburner.com/co/TmxG"

  # ask user to enter the city
  get_city = input('도시 이름: ').lower()
  print('OK! {}. 잠시만 기다려주세요...'.format(get_city))
  
  # set the value to the corresponding xml url
  if get_city == '서울' or get_city == 'seoul':
    get_city = seoul
  elif get_city == '강원' or get_city == 'gangwon':
    get_city = gangwon
  elif get_city == '경기' or get_city == 'gyeonggi':
    get_city = gyeonggi
  elif get_city == '경남' or get_city == 'gyeongnam':
    get_city = gyeongnam
  elif get_city == '경북' or get_city == 'gyeongbuk':
    get_city = gyeongbuk
  elif get_city == '광주' or get_city == 'gwangju':
    get_city = gwangju
  elif get_city == '대구' or get_city == 'daegu':
    get_city = daegu
  elif get_city == '대전' or get_city == 'daejeon':
    get_city = daejeon
  elif get_city == '부산' or get_city == 'busan':
    get_city = busan
  elif get_city == '울산' or get_city == 'ulsan':
    get_city = ulsan
  elif get_city == '인천' or get_city == 'incheon':
    get_city = incheon
  elif get_city == '전남' or get_city == 'jeonnam':
    get_city = jeonnam
  elif get_city == '전북' or get_city == 'jeonbuk':
    get_city = jeonbuk
  elif get_city == '제주' or get_city == 'jeju':
    get_city = jeju
  elif get_city == '충남' or get_city == 'chungnam':
    get_city = chungnam
  elif get_city == '충북' or get_city == 'chungbuk':
    get_city = chungbuk
  elif get_city == '전국' or get_city == 'national':
    get_city = national
  else:
    print('미안 난 이해를 못 했어...')
    start()

  # parse RSS feed
  feed = feedparser.parse(get_city)
  feed_title = feed['feed']['title']
  feed_entries = feed.entries
  
  # set voice settings
  voice = texttospeech.types.VoiceSelectionParams(
      language_code='ko-KR',
      name='ko-KR-Wavenet-A',
      ssml_gender=texttospeech.enums.SsmlVoiceGender.FEMALE)

  audio_config = texttospeech.types.AudioConfig(
      audio_encoding=texttospeech.enums.AudioEncoding.MP3,
      speaking_rate=0.6)
  
  # Create directory
  dst = '/content/audio/'
  if not os.path.exists(dst):
    os.makedirs(dst)

  # read entries
  i=1
  for entry in feed.entries:
    article_title = entry.title
    article_description = entry.description
    article_link = entry.link
    
    clean_title = re.sub("-", "", article_title)
    clean_description = re.sub("<.*?>", "", article_description)
    clean_description2 = re.sub(":", "", clean_description)
    clean_description3 = re.sub("·", " ", clean_description2)
    clean_description4 = re.sub("-", "", clean_description3)
    
    print("\n {} {}".format(clean_title, clean_description4))
    
    # synthesise text to speech
    input_text = texttospeech.types.SynthesisInput(text=clean_title+clean_description4)
    response = client.synthesize_speech(input_text, voice, audio_config)
    
    

    # The response's audio_content is binary.
    with open('listing_{}.mp3'.format(i), 'wb') as out:
        out.write(response.audio_content)
        display(Audio('listing_{}.mp3'.format(i),rate=44100))
        
        src = "/content/"
        shutil.move(os.path.join(src, "listing_{}.mp3".format(i)), os.path.join(dst, "listing_{}.mp3".format(i)))
        #print('Audio content written to file "output.mp3"')
                
    i=i+1
    
print('DoReMi에서 오신 것을 환영합니다!')
start()

#for f in glob.iglob("/content/audio/*"):
#  display(Audio(f,rate=44100))

DoReMi에서 오신 것을 환영합니다!
도시 이름: seoul
OK! seoul. 잠시만 기다려주세요...

 (주)서현운수  이마트배송기사모집/기본급285+인센(월420)/초보가능  회사명  (주)서현운수 모집내용  (주)서현운수  이마트배송기사모집/기본급285+인센(월420)/초보가능 직종   근무형태  계약직, 파견직 경력  무관 학력  무관 지역  서울, 경기 마감일  수시채용 입사지원하기 (주)서현운수 기업의 진행중인 채용정보 보기



 더휴컴퍼니  UGIZ 영업MD 경력사원 채용  회사명  더휴컴퍼니 모집내용  더휴컴퍼니  UGIZ 영업MD 경력사원 채용 직종   근무형태  정규직 경력  경력 학력  전졸(예) 지역  서울 마감일  06/22(토) 입사지원하기 더휴컴퍼니 기업의 진행중인 채용정보 보기



 더휴컴퍼니  UGIZ 영업팀 경력사원 모집  회사명  더휴컴퍼니 모집내용  더휴컴퍼니  UGIZ 영업팀 경력사원 모집  직종   근무형태  정규직 경력  경력 학력  무관 지역  서울 마감일  06/22(토) 입사지원하기 더휴컴퍼니 기업의 진행중인 채용정보 보기



 동현종합물류  14톤윙바디 화물차 운전기사님을 모집합니다/월250만  회사명  동현종합물류 모집내용  동현종합물류  14톤윙바디 화물차 운전기사님을 모집합니다/월250만 직종   근무형태  정규직, 계약직 경력  무관 학력  무관 지역  서울, 경기, 충북 마감일  수시채용 입사지원하기 동현종합물류 기업의 진행중인 채용정보 보기



 (주)롯데TLS  운전/1톤투잡600만원/5톤제주도왕복/.1000만원순수입  회사명  (주)롯데TLS 모집내용  (주)롯데TLS  운전/1톤투잡600만원/5톤제주도왕복/.1000만원순수입 직종   근무형태  정규직, 계약직, 파견직, 전 체 경력  무관 학력  무관 지역  서울, 경기, 인천, 경기, 경기, 경기, 경기, 경기 마감일  수시채용 입사지원하기 (주)롯데TLS 기업의 진행중인 채용정보 보기



 ㈜위메프  (MD)비즈상품 경력MD(팀장급 포함)  회사명  ㈜위메프 모집내용  ㈜위메프  (MD)비즈상품 경력MD(팀장급 포함) 직종   근무형태  정규직 경력  경력 학력  무관 지역  서울 마감일  06/19(수) 입사지원하기 ㈜위메프 기업의 진행중인 채용정보 보기



 패스트파이브  인테리어 디자이너 채용 (경력)  회사명  패스트파이브 모집내용  패스트파이브  인테리어 디자이너 채용 (경력) 직종   근무형태  정규직 경력  경력 학력  대졸(예) 지역  서울 마감일  06/14(금) 입사지원하기 패스트파이브 기업의 진행중인 채용정보 보기



 서강물류  주5일.월금/대기업 LG이노텍/월수입600/수도권경상권 윙바디 지입운송/초보자가능/전액할부가능  회사명  서강물류 모집내용  서강물류  주5일.월금/대기업 LG이노텍/월수입600/수도권경상권 윙바디 지입운송/초보자가능/전액할부가능 직종   근무형태  정규직 경력  무관 학력  무관 지역  서울, 서울, 경기, 경기, 경남, 경남, 경북, 경북 마감일  채용시까지 입사지원하기 서강물류 기업의 진행중인 채용정보 보기



 (주)티몬  (티몬) DB엔지니어 채용( DBA)  회사명  (주)티몬 모집내용  (주)티몬  (티몬) DB엔지니어 채용( DBA) 직종   근무형태  정규직 경력  경력 학력  전졸(예) 지역  서울 마감일  06/05(수) 입사지원하기 (주)티몬 기업의 진행중인 채용정보 보기



 오스템임플란트(주)  시설관리 기계 담당 모집  회사명  오스템임플란트(주) 모집내용  오스템임플란트(주)  시설관리 기계 담당 모집 직종   근무형태  정규직 경력  경력 학력  전졸(예) 지역  서울 마감일  05/26(일) 입사지원하기 오스템임플란트(주) 기업의 진행중인 채용정보 보기
